Импортируем модули и данные

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale

X_train = pd.read_csv('data/features.csv')
X_test = pd.read_csv('data/features_test.csv')
y = X_train['radiant_win']

## Градиентный бустинг
**1.** Удаляем признаки, связанные с исходом матча

In [2]:
X_train = X_train.drop([
        'tower_status_radiant',
        'duration',
        'tower_status_dire',
        'radiant_win',
        'barracks_status_dire',
        'barracks_status_radiant'
    ], axis=1)

**2.** Проверяем выборку на наличие пропусков в данных и выводим признаки, данные для которых пропущены

In [3]:
X_all = pd.concat((X_train, X_test), axis=0)
nan_stats = X_all.isnull().sum()
features_with_nans = dict(filter(lambda f: f[1] > 0, list(nan_stats.items())))

print(', '.join(features_with_nans.keys()))
print('Average missings in features with missing data', np.average(list(features_with_nans.values())))

dire_bottle_time, first_blood_player1, radiant_courier_time, dire_courier_time, radiant_bottle_time, radiant_first_ward_time, dire_flying_courier_time, dire_first_ward_time, first_blood_team, first_blood_time, first_blood_player2, radiant_flying_courier_time
Average missings in features with missing data 18958.75


Данные пропущены в следующих столбцах:
```
    first_blood_player1,
    radiant_flying_courier_time,
    first_blood_team,
    dire_first_ward_time,
    dire_courier_time,
    radiant_courier_time,
    first_blood_time,
    dire_bottle_time,
    radiant_first_ward_time,
    radiant_bottle_time,
    first_blood_player2,
    dire_flying_courier_time
```
Всего 12 столбцов с пропущенными данными. В среднем в этих 12 столбцах пропущено по 18958.75 значений (или ~16.5%). Это прилично, но вполне терпимо.

Так как наши данные собраны за первые 5 минут матча, то может оказаться так, что некоторые события за этот интервал еще не произошли (например, **radiant_courier_time** или **dire_bottle_time**) произошли в другой момент времени. А некоторые события (даже **first_blood_team**) теоретически могут не случиться вообще (в случае первой крови такое обычно возможно только при "китайской" доте 1 на 1 с мочиловом бараков, а не друг друга).
   

**3.** Заполняем пропуски

In [4]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

**4.** Целевая переменная (исход матча) лежит в столбце **radiant_win**. Мы уже положили ее в переменную `y` на первом шаге (потому что потом дропали столбцы в X_train)

**5.** Обучаем нашу модель! Пробуем количество деревьев от 10 до 50 с шагом в 10

In [5]:
cv = KFold(len(X_train), shuffle=True, n_folds=5, random_state=666)
n_estimators_variants = np.linspace(10, 50, num=5)
n_estimators_scores = list(range(len(n_estimators_variants)))

for i, n_estimators in enumerate(n_estimators_variants):
    model = GradientBoostingClassifier(n_estimators=int(n_estimators), random_state=666)
    scores = cross_val_score(model, X_train, y=y, cv=cv, scoring='roc_auc')
    n_estimators_scores[i] = scores.mean()
    print('Model created. N trees is:', n_estimators, 'Score is:', n_estimators_scores[i])

print('Estimator scores:', ', '.join(map(str, n_estimators_scores)))

Model created. N trees is: 10.0 Score is: 0.664686904581
Model created. N trees is: 20.0 Score is: 0.682239421827
Model created. N trees is: 30.0 Score is: 0.689557054588
Model created. N trees is: 40.0 Score is: 0.694170033075
Model created. N trees is: 50.0 Score is: 0.697536891311
Estimator scores: 0.664686904581, 0.682239421827, 0.689557054588, 0.694170033075, 0.697536891311


Классификаторы обучались достаточно быстро: для 30 деревьев — около 5 минут.
На кросс-валидации для 30 деревьев получилось качество AUC-ROC **~0.69**. Если увеличить количество деревьев, то оно будет немного выше: на 50 деревьях **~0.7**. Чтобы ускорить работу алгоритма, можно распараллелить обучение (как это делается в XGBoost) или попробовать методы уменьшения размерности. Также подходят советы, которые дали в инструкции: сократить глубину дерева или проверяться на подмножестве объектов.

## Логистическая регрессия
**1.** Обучаем модель и подбираем коэффициент регуляризации. Оформим это в виде функции, потому что нам это еще раз предстоит делать. Не забываем перемасштабировать признаки для логистической регрессии.

In [6]:
def train_logistic_regression(X_train, y):
    X_train_scaled = scale(X_train)
    cv = KFold(len(X_train), shuffle=True, n_folds=5, random_state=666)
    C_variants = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
    C_scores = list(range(len(C_variants)))
    for i, C in enumerate(C_variants):
        model = LogisticRegression(penalty='l2', C=C, random_state=666)
        scores = cross_val_score(model, X_train_scaled, y=y, cv=cv, scoring='roc_auc')
        C_scores[i] = scores.mean()
        print('Model created. C value is:', C, 'Score is:', C_scores[i])
    best_score = max(C_scores)
    best_C = C_variants[ C_scores.index(best_score) ]
    return best_score, best_C

best_score, best_C = train_logistic_regression(X_train, y)
print("Best C value:", best_C, 'Score is:', best_score)

/Users/universome/dev/courses/intro-into-ml/lib/python3.5/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


Model created. C value is: 0.0001 Score is: 0.711357235409
Model created. C value is: 0.001 Score is: 0.71634178315
Model created. C value is: 0.01 Score is: 0.716523079975
Model created. C value is: 0.1 Score is: 0.716499294762
Model created. C value is: 1 Score is: 0.716495414865
Model created. C value is: 10 Score is: 0.716495275724
Model created. C value is: 100 Score is: 0.71649516559
Model created. C value is: 1000 Score is: 0.716495115813
Best C value: 0.01


Логистическая регрессия обучается намного быстрее бустинга и ее качество немного выше: **~0.715** при оптимальном параметре C, равным **0.01**.

Повышение качества у логистической регрессии может быть обусловлено двумя причинами:

1. Выборка действительно разделима "линейной" гиперплоскостью (которую легко находит логистическая регрессия), которая при этом не является параллельной координатным осям (которую строят решающие деревья)

2. Мы никак не учитывали возможность переобучения решающего леса, поэтому он может дать качество получше, если попробовать прюнинг

[1] https://www.quora.com/What-are-the-advantages-of-logistic-regression-over-decision-trees

**2.** Удаляем категориальные признаки из выборки и повторяем предыдущую процедуру.

In [7]:
X_train = X_train.drop([
        'lobby_type',
        'r1_hero',
        'r2_hero',
        'r3_hero',
        'r4_hero',
        'r5_hero',
        'd1_hero',
        'd2_hero',
        'd3_hero',
        'd4_hero',
        'd5_hero'], axis=1)

In [8]:
best_score, best_C = train_logistic_regression(X_train, y)
print("Best C value:", best_C, 'Score is:', best_score)

/Users/universome/dev/courses/intro-into-ml/lib/python3.5/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


Model created. C value is: 0.0001 Score is: 0.711322562411
Model created. C value is: 0.001 Score is: 0.716346505099
Model created. C value is: 0.01 Score is: 0.716528826331
Model created. C value is: 0.1 Score is: 0.716504145319
Model created. C value is: 1 Score is: 0.716500102191
Model created. C value is: 10 Score is: 0.71650001162
Model created. C value is: 100 Score is: 0.716499903472
Model created. C value is: 1000 Score is: 0.71649991299
Best C value: 0.01


После удаления категориальных признаков качество осталось на прежнем уровне (даже на тысячные доли ухудшилось). Значит, они практически не влияли на построение предсказания.

**3.** Количество уникальных героев в матчах есть в файле `heroes.csv` Оно равно **113**.

**4.** Теперь нужно преобразовать категориальные признаки для героев по принципу, похожему на бинаризацию, только кодирование троичное. Оформим это в виде функции, потому что нам потом еще для тестовой выборки это делать.

In [10]:
X_train = pd.read_csv('data/features.csv')
X_train = X_train.drop([
        'tower_status_radiant',
        'duration',
        'tower_status_dire',
        'radiant_win',
        'barracks_status_dire',
        'barracks_status_radiant',
        'lobby_type'
    ], axis=1)
X_train = X_train.fillna(0)

def get_hero_features(data):
    heros_amount = 113
    X_pick = np.zeros((data.shape[0], heros_amount))
    for i, match_id in enumerate(data.index):
        for p in range(5):
            X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
    X_pick_df = pd.DataFrame(X_pick, columns=['hero_' + str(n) for n in range(1, heros_amount + 1)])
    return X_pick_df

X_train = pd.concat((X_train, get_hero_features(X_train)), axis=1)

In [11]:
best_score, best_C = train_logistic_regression(X_train, y)
print("Best score:", best_score)

/Users/universome/dev/courses/intro-into-ml/lib/python3.5/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


Model created. C value is: 0.0001 Score is: 0.74289645091
Model created. C value is: 0.001 Score is: 0.751711403101
Model created. C value is: 0.01 Score is: 0.752020095674
Model created. C value is: 0.1 Score is: 0.751986236407
Model created. C value is: 1 Score is: 0.751979706802
Model created. C value is: 10 Score is: 0.751979235186
Model created. C value is: 100 Score is: 0.751979135531
Model created. C value is: 1000 Score is: 0.751979146107
Best score: 0.752020095674


Как видим, качество значительно подросло — до **~0.752**. Это закономерный факт: раньше алгоритм пытался разделить выборку на два класса в пространстве, значения которого были от 1 до 113, воспринимая эти значения как непрерывную величину. Разумеется, это достаточно абсурдно, потому что значения категориальны и простой линией их не разделишь. Теперь алгоритм рисует линии в этих 113 пространствах, значения которых принадлежат множеству {-1,0,1} — это намного логичнее и проще.

**5.** Сделаем предсказания для тестовой выборки

In [23]:
clf = LogisticRegression(penalty='l2', C=0.01, random_state=666)
clf.fit(scale(X_train), y)

X_test = pd.read_csv('data/features_test.csv')
X_test = X_test.drop('lobby_type', axis=1)
X_test = X_test.fillna(0)
X_test = pd.concat((X_test, get_hero_features(X_test)), axis=1)

predictions = clf.predict_proba(scale(X_test))
print('Минимальное значение предсказания:', predictions.min())
print('Максимальное значение предсказания:', predictions.max())

/Users/universome/dev/courses/intro-into-ml/lib/python3.5/site-packages/sklearn/preprocessing/data.py:167: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


Минимальное значение предсказания: 0.00359114962028
Максимальное значение предсказания: 0.99640885038


Минимальное значение предсказания равно **~0.0036**, а максимальное — **~0.99**.